# Лабораторная работа № 4.1
## "Определение энергии $\alpha$-частиц по величине их пробега в воздухе"

Измеряется пробег $\alpha$-частиц в воздухе двумя способами $-$ с помощью торцевого счетчика Гейгера и сцинтилляционного счетчика, $-$ и по полученным величинам определяется их энергия.

\textit{Могут ли лабы подгонять себя сами?} 